<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-the-libraries" data-toc-modified-id="Load-the-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Train Test Split</a></span></li><li><span><a href="#Pure-Premium-Modelling-:-Tweedie-GLM" data-toc-modified-id="Pure-Premium-Modelling-:-Tweedie-GLM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pure Premium Modelling : Tweedie GLM</a></span></li><li><span><a href="#Modelling:-Product-of-Frequency-and-Severity-Modelling" data-toc-modified-id="Modelling:-Product-of-Frequency-and-Severity-Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling: Product of Frequency and Severity Modelling</a></span></li><li><span><a href="#Lorentz-Curves" data-toc-modified-id="Lorentz-Curves-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Lorentz Curves</a></span></li></ul></div>

# Colab

In [1]:
%%capture
# capture will not print in notebook

import os
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    ## install modules
    !pip install watermark
    !pip install scikit-plot
    !pip install -U scikit-learn # tweedie needs > 0.23

    ## print
    print('Environment: Google Colaboratory.')

# NOTE: If we update modules in gcolab, we need to restart runtime.

In [2]:
import scikitplot as skplot

In [3]:
TREE_METHOD = 'auto'
try:
    import tensorflow as tf
    has_gpu = tf.test.gpu_device_name()
    TREE_METHOD = 'gpu_hist' if has_gpu else 'auto'
except:
    TREE_METHOD = 'auto'

print(TREE_METHOD)

auto


In [4]:
!pwd
!ls

/content
sample_data


In [5]:
if ENV_COLAB:
    # raw
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv?raw=true"
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/raw/freMTPL2sev.csv?raw=true"

    # processed
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/clean_data.csv.zip?raw=true"
    !wget -c "https://raw.githubusercontent.com/bhishanpdl/Datasets/master/Projects/French_Motor_Claims/processed/features.json"
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/X.csv.zip?raw=true"
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/X.npz?raw=true"


--2020-09-27 15:07:11--  https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/bhishanpdl/Datasets/raw/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv [following]
--2020-09-27 15:07:11--  https://github.com/bhishanpdl/Datasets/raw/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bhishanpdl/Datasets/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv [following]
--2020-09-27 15:07:11--  https://raw.githubusercontent.com/bhishanpdl/Datasets/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 

In [6]:
!ls

'clean_data.csv.zip?raw=true'  'freMTPL2sev.csv?raw=true'  'X.npz?raw=true'
 features.json		        sample_data
'freMTPL2freq.csv?raw=true'    'X.csv.zip?raw=true'


In [7]:
if ENV_COLAB:

    # clean old dir
    !rm -rf ../data

    !mkdir -p ../data/processed
    !mkdir -p ../data/raw
    !mkdir -p ../outputs

    !mv freMTPL2freq.csv?raw=true ../data/raw/freMTPL2freq.csv
    !mv freMTPL2sev.csv?raw=true ../data/raw/freMTPL2sev.csv

    !mv features.json ../data/processed/features.json
    !mv clean_data.csv.zip?raw=true ../data/processed/clean_data.csv.zip
    !mv X.csv.zip?raw=true ../data/processed/X.csv.zip
    !mv X.npz?raw=true ../data/processed/X.npz

In [8]:
!ls

sample_data


In [9]:
!ls ../data/raw

freMTPL2freq.csv  freMTPL2sev.csv


In [10]:
!ls ../data/processed

clean_data.csv.zip  features.json  X.csv.zip  X.npz


# Load the libraries

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import os,sys,time
import sklearn
import scipy
import matplotlib.pyplot as plt
sns.set()

import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_tweedie_deviance
from sklearn.metrics import auc

SEED = 100
pd.set_option('max_columns',100)
pd.set_option('plotting.backend','matplotlib') # matplotlib, bokeh, altair, plotly
%load_ext watermark
%watermark -iv

numpy      1.18.5
pandas     1.0.5
scikitplot 0.3.7
seaborn    0.10.1
sklearn    0.23.2
scipy      1.4.1
joblib     0.16.0
json       2.0.9
tensorflow 2.3.0



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load the data

In [12]:
%%bash
pwd

/content


In [13]:
ifile = '../data/processed/clean_data.csv.zip'
# ifile = "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/clean_data.csv.zip?raw=true"

In [14]:
df = pd.read_csv(ifile, compression='zip')
print(df.shape)
df.head(2).append(df.tail(2))

(100000, 15)


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
1,0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
99998,0,0.90,C,7,9,44,50,B1,Regular,191,R24,0.0,0.0,0.0,0.0
99999,0,0.90,E,4,12,53,50,B1,Regular,4116,R24,0.0,0.0,0.0,0.0


In [15]:
X = scipy.sparse.load_npz("../data/processed/X.npz")
X.shape, type(X)

((100000, 71), scipy.sparse.csr.csr_matrix)

In [16]:
# ifile = '../data/processed/X.csv.zip'
# df_X = pd.read_csv(ifile, compression='zip')
# print(df_X.shape)
# df_X.head(2).append(df_X.tail(2))

# columns
"""
x0_B1	x0_B10	x0_B11	x0_B12	x0_B13	x0_B14	x0_B2	x0_B3	x0_B4	x0_B5
x0_B6	x1_4	x1_5	x1_6	x1_7	x1_8	x1_9	x1_10	x1_11	x1_12
x1_13	x1_14	x1_15	x2_Diesel	x2_Regular	x3_R11	x3_R21	x3_R22	x3_R23
x3_R24	x3_R25	x3_R26	x3_R31	x3_R41	x3_R42	x3_R43	x3_R52	x3_R53	x3_R54
x3_R72	x3_R73	x3_R74	x3_R82	x3_R83	x3_R91	x3_R93	x3_R94	x4_A	x4_B
x4_C	x4_D	x4_E	x4_F	VehAge_0	VehAge_1	VehAge_2	VehAge_3
VehAge_4	VehAge_5	VehAge_6	VehAge_7	DrivAge_0	DrivAge_1	DrivAge_2	
DrivAge_3	DrivAge_4	DrivAge_5	DrivAge_6	DrivAge_7	Density	BonusMalus

X only have transformed version of these columns
cols_ohe_before = ["VehBrand", "VehPower", "VehGas", "Region", "Area"]
cols_kbin_before = ["VehAge", "DrivAge"]
cols_log_scale = ["Density"]
cols_pass =  ["BonusMalus"]
""";

In [17]:
df.head(2)

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
1,0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0


In [18]:
np.array(X[0].todense())[0][-5:] # last elements of first row

array([ 0.        ,  1.        ,  0.        ,  0.69864446, 50.        ])

In [19]:
with open("../data/processed/features.json") as fi:
    json_features = json.load(fi)

In [20]:
json_features.keys()

dict_keys(['cols_ohe_before', 'cols_kbin', 'cols_log_scale', 'cols_pass', 'feature_names_before', 'feature_names_after', 'desc'])

# Train Test Split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
df_train, df_test, X_train, X_test = train_test_split(df, X, random_state=SEED)

df_train.shape, df_test.shape, X_train.shape, X_test.shape

((75000, 15), (25000, 15), (75000, 71), (25000, 71))

# Modelling: Xgboost Tweedie regressor

In [23]:
import xgboost as xgb

In [24]:
target = 'PurePremium'

y_train = df_train[target].to_numpy().ravel()
y_test = df_test[target].to_numpy().ravel()

dtrain = xgb.DMatrix(data=X_train.todense(),label=y_train)
dtest = xgb.DMatrix(data=X_test.todense(), label=y_test)

In [25]:
# apply offsets
# Ref: https://towardsdatascience.com/offsetting-the-model-logic-to-implementation-7e333bc25798
# there are different ways to apply offsets for different modules.
# statsmodel: exposure
# xgboost: set_base_margin with log
# lightgbm: set_init_score()
dtrain.set_base_margin(np.log(df_train['Exposure'].to_numpy()))
dtest.set_base_margin(np.log(df_test['Exposure'].to_numpy()))

In [26]:
%%time
params = {
    'objective':'reg:tweedie',
    'colsample_bytree': 1.0,
    'learning_rate': 0.01,
    'gamma':1.5,
    'max_depth': 2,
    'subsample':0.6, 
    'reg_alpha': 0,
    'reg_lambda':1,
    'min_child_weight':5,
    'n_estimators':2000,
    'tweedie_variance_power':1.9}

model_xgb = xgb.train(params=params, dtrain=dtrain, num_boost_round=1000)

CPU times: user 3min 2s, sys: 250 ms, total: 3min 2s
Wall time: 1min 32s


In [27]:
tr_preds = model_xgb.predict(dtrain)
tx_preds = model_xgb.predict(dtest)

In [28]:
tr_mae = mean_absolute_error(y_train,tr_preds)
tx_mae = mean_absolute_error(y_test,tx_preds)

tr_mse = mean_squared_error(y_train, tr_preds)
tx_mse = mean_squared_error(y_test,tx_preds)

df_eval_twd = pd.DataFrame(
{'train': [np.nan, tr_mae, tr_mse],
'test': [np.nan, tx_mae, tx_mse]})

df_eval_twd.index = ['D2','mean_absolute_error','mean_squared_error']
df_eval_twd

,train,test
D2,NaN,NaN
mean_absolute_error,1.760538e+03,1.588351e+03
mean_squared_error,1.481952e+09,1.659363e+08


In [29]:
%%time
cv_results = xgb.cv(dtrain=dtrain, params=params, nfold=3,
                    num_boost_round=2000,
                    early_stopping_rounds=10,
                    metrics="rmse",
                    as_pandas=True,
                    seed=SEED)

CPU times: user 3min 13s, sys: 341 ms, total: 3min 13s
Wall time: 1min 38s


In [30]:
cv_results.shape

(448, 4)

In [31]:
cv_results.tail()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
443,34847.445312,16248.746864,28785.776855,25490.221577
444,34847.441732,16248.751926,28785.770670,25490.226078
445,34847.438151,16248.754184,28785.763672,25490.228446
446,34847.434245,16248.756901,28785.758463,25490.232558
447,34847.427409,16248.758325,28785.754232,25490.235459
